In [ ]:
!pip install py3langid

In [ ]:
import pandas as pd
import nltk
import py3langid as langid
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.probability import FreqDist
import pickle

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("names")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('tagsets')
nltk.download('tagsets_json')
nltk.download('punkt_tab')

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
names = nltk.corpus.names.words()

In [ ]:
df = pd.read_csv("restaurant_reviews.csv")
df

In [ ]:
df = df[df["Rating"] != "Like"]

In [ ]:
df = df[["Review", "Rating", "Time"]]

In [ ]:
df["Rating"] = df["Rating"].astype("float64")

In [ ]:
df = df[~df["Review"].isna()]

In [ ]:
df.info()

In [ ]:
df["Rating"].hist()

In [ ]:
df["Time"] = pd.to_datetime(df["Time"])

In [ ]:
df["Hour"] = df["Time"].dt.hour

In [ ]:
df["Weekday"] = df["Time"].dt.weekday

In [ ]:
df["Month"] = df["Time"].dt.month

In [ ]:
df

In [ ]:
df.boxplot("Rating", by="Month")

In [ ]:
df["Language"] = df["Review"].apply(lambda x: langid.classify(x)[0])
df

In [ ]:
df = df[df["Language"] == "en"]
df

In [ ]:
df["Rank"] = df["Rating"].rank(method="first")

In [ ]:
df["Quartile"] = pd.qcut(df["Rank"], 4, labels=[1, 2, 3, 4])

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [ ]:
tokens_unwanted = []

In [ ]:
def get_tokens(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    if token in tokens_unwanted: continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    cleaned_tokens.append(token)

  cleaned_tokens = [i[0] for i in pos_tag(cleaned_tokens) if i[1][:2] in ["NN", "VB", "RB", "JJ"]]

  return cleaned_tokens


In [ ]:
full_vocabulary = []
vocabulary = {category:[] for category in df["Quartile"].unique()}
for _, row in df.iterrows():
  tokens = get_tokens(row["Review"])
  full_vocabulary += tokens
  vocabulary[row["Quartile"]] += tokens

In [ ]:
FreqDist(full_vocabulary).most_common(20)

In [ ]:
FreqDist(vocabulary[4]).most_common(20)

In [ ]:
most_common_tokens = set()

for quartile in df["Quartile"].unique():
  most_common_quartile_tokens = [i[0] for i in FreqDist(vocabulary[quartile]).most_common(250)]
  most_common_tokens = most_common_tokens.union(set(most_common_quartile_tokens))

In [ ]:
print(len(list(most_common_tokens)))

In [ ]:
with open(f"most_common.tokens", "wb") as file:
  pickle.dump(most_common_tokens, file)

In [ ]:
def get_tokens_training(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    if token not in most_common_tokens: continue
    cleaned_tokens.append(token)

  return " ".join(cleaned_tokens)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df["Tokens String"] = df["Review"].apply(lambda x: get_tokens_training(x))
df

In [ ]:
vectorizer = TfidfVectorizer(vocabulary=most_common_tokens)
X = vectorizer.fit_transform(df["Tokens String"])

In [ ]:
with open("vectorizer.pkl", "wb") as file:
  pickle.dump(vectorizer, file)

In [ ]:
y = df["Rating"]

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train

In [ ]:
model = AdaBoostRegressor()

In [ ]:
model.fit(X_train.toarray(), y_train)

In [ ]:
model.predict(X_test.toarray())

In [ ]:
mean_absolute_error(y_test, model.predict(X_test.toarray()))

# MAE = 1.19 then in average model garantees to differ by +- 1.19
# For example: 5 has a lower bound 3.81 and an upper bound 6.19

In [ ]:
mean_squared_error(y_test, model.predict(X_test.toarray()))

In [ ]:
r2_score(y_test, model.predict(X_test.toarray()))

In [ ]:
with open("model.pkl", "wb") as file:
  pickle.dump(model, file)

##"Modelo en producción"

In [ ]:
import pandas as pd
import nltk
import py3langid as langid
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.probability import FreqDist
import pickle
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
with open("model.pkl", "rb") as file:
  loaded_model = pickle.load(file)

with open("vectorizer.pkl", "rb") as file:
  loaded_vectorizer = pickle.load(file)

with open("most_common.tokens", "rb") as file:
  loaded_mct = pickle.load(file)

In [ ]:
nltk.download("stopwords")
nltk.download("names")

In [ ]:
lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words("english")
names = nltk.corpus.names.words()

In [ ]:
def get_tokens_predict(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    if token not in loaded_mct: continue
    cleaned_tokens.append(token)

  return " ".join(cleaned_tokens)


In [ ]:
predict_df = pd.read_csv("restaurant_reviews.csv")

In [ ]:
predict_df = predict_df[~predict_df["Review"].isna()]
predict_df = predict_df.reset_index()

In [ ]:
predict_df["Tokens String"] = predict_df["Review"].apply(lambda x: get_tokens_predict(x))

In [ ]:
X = loaded_vectorizer.transform(predict_df["Tokens String"])

In [ ]:
predictions = loaded_model.predict(X.toarray())

In [ ]:
pd.concat([predict_df, pd.Series(predictions, name="Prediction")], axis=1)